In [1]:
import polars as pl
import pandas as pd
import plotly.express as px 
import requests
from datetime import datetime
from pathlib import Path
from typing import List, Dict
import os

In [2]:
# constantes
MUSEO_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv"
CINES_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/f7a8edb8-9208-41b0-8f19-d72811dcea97/download/salas_cine.csv"
ESPACIOS_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/11_bibliotecapopular-datos-abiertos.csv"

list_dicts_url=[
 {
 "name":"museo", "url":MUSEO_URL    
 },
 {
 "name":"cines", "url":CINES_URL    
 },
 {
 "name":"espacios", "url":ESPACIOS_URL    
 }
]

# Path
BASE_FILE_DIR = Path("C:/Users/joan1/OneDrive/Documentos/Programacion/Python/Analisis de datos publicos ARG/tmp").resolve()

# tricks for iterate with a str path (Yuo can define in places holders)
file_path_crib = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv"

norm_cols= [
    "cod_localidad",
    "id_provincia",
    "id_departamento",
    "categoría",
    "provincia",
    "localidad",
    "nombre",
    "domicilio",
    "código postal",
    "número de teléfono",
    "mail",
    "web"
]

In [7]:
datetime.now().strftime("%Y-%m-%d")

'2023-10-22'

### Funtions

In [8]:
def get_response_and_create_csv(data_name:dict) -> pl.DataFrame:
    """
        Get the response from the url and create a csv file with the data
    """
    r=requests.get(
        url=data_name["url"]
    )
    r.encoding= "utf-8"

    now = datetime.now()
    category= data_name["name"]

    # create files and fill the places holders
    file_path = BASE_FILE_DIR.joinpath(file_path_crib.format(category=category, 
                                                             year=now.year, 
                                                             month=now.month, 
                                                             day=now.day))
    # create the file
    file_path.parent.mkdir(parents=True, exist_ok=True)

    with open(file=file_path, mode="w") as f:
        f.write(r.text)

    return pd.read_csv(file_path, encoding="latin-1").reset_index()

def columns_rename(df:pd.DataFrame, old_colums, new_columns):
    """
        Exchange the old columns names for the new columns names
    """
    list_changes = {old:new for old, new in zip(old_colums, new_columns)}
    print(list_changes)

    try:
        df = df.rename(columns=list_changes) 
    except Exception:
        print("The columns names not exist")
    return df


### Peticion get para obtener datos

In [9]:
# # r=requests.get("https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_4207def0-2ff7-41d5-9095-d42ae8207a5d")
# r=requests.get("https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv")
# r.encoding= "utf-8"

In [10]:
# with open("open","w") as f_out: 
#     f_out.write(r.text)

In [11]:
# pd.read_csv("open", delimiter=";", decimal=",",encoding="utf-8")
#pd.read_csv("open",encoding="utf-8-sig")
# pd.read_csv("open",encoding="latin-1")

In [12]:
# pl.read_csv("open", encoding= "latin-1")

### museo

In [13]:
df_museo = get_response_and_create_csv(list_dicts_url[0])

In [14]:
df_museo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                1182 non-null   int64  
 1   Cod_Loc              1182 non-null   int64  
 2   IdProvincia          1182 non-null   int64  
 3   IdDepartamento       1182 non-null   int64  
 4   Observaciones        0 non-null      float64
 5   categoria            1182 non-null   object 
 6   subcategoria         1049 non-null   object 
 7   provincia            1182 non-null   object 
 8   localidad            1182 non-null   object 
 9   nombre               1182 non-null   object 
 10  direccion            1164 non-null   object 
 11  piso                 0 non-null      float64
 12  CP                   1116 non-null   object 
 13  cod_area             989 non-null    float64
 14  telefono             988 non-null    object 
 15  Mail                 264 non-null    o

In [15]:
df_museo.columns

Index(['index', 'Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'categoria', 'subcategoria', 'provincia', 'localidad', 'nombre',
       'direccion', 'piso', 'CP', 'cod_area', 'telefono', 'Mail', 'Web',
       'Latitud', 'Longitud', 'TipoLatitudLongitud', 'Info_adicional',
       'fuente', 'jurisdiccion', 'año_inauguracion', 'actualizacion'],
      dtype='object')

In [16]:
new_columns = [i for i in norm_cols if i not in df_museo.columns]
new_columns

['cod_localidad',
 'id_provincia',
 'id_departamento',
 'categoría',
 'domicilio',
 'código postal',
 'número de teléfono',
 'mail',
 'web']

In [17]:
old_columns = [
'Cod_Loc', 'IdProvincia',
 'IdDepartamento',
 'categoria',
 'direccion',
 'CP', 
 'telefono',
 'Mail', 
 'Web',
 ]

In [18]:
df_museo=columns_rename(df_museo, 
                        old_columns,
                        new_columns 
                        )
df_museo.columns

{'Cod_Loc': 'cod_localidad', 'IdProvincia': 'id_provincia', 'IdDepartamento': 'id_departamento', 'categoria': 'categoría', 'direccion': 'domicilio', 'CP': 'código postal', 'telefono': 'número de teléfono', 'Mail': 'mail', 'Web': 'web'}


Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'Observaciones', 'categoría', 'subcategoria', 'provincia', 'localidad',
       'nombre', 'domicilio', 'piso', 'código postal', 'cod_area',
       'número de teléfono', 'mail', 'web', 'Latitud', 'Longitud',
       'TipoLatitudLongitud', 'Info_adicional', 'fuente', 'jurisdiccion',
       'año_inauguracion', 'actualizacion'],
      dtype='object')

In [19]:
df_museo[new_columns]

,cod_localidad,id_provincia,id_departamento,categoría,domicilio,código postal,número de teléfono,mail,web
0,6588100,6,6588,Espacios de Exhibición Patrimonial,Libertad 1191,B6500EVL,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar
1,6077010,6,6077,Espacios de Exhibición Patrimonial,Gerardo Risso y España,B2740FMJ,452 931,museoarrecifes@hotmail.com,NaN
2,6035010,6,6035,Espacios de Exhibición Patrimonial,Calle 11 (Av. San Martín) Nº 830,S3561AKT,481 200,NaN,NaN
3,6035010,6,6035,Espacios de Exhibición Patrimonial,"Sarmiento 101, 1 º Piso",B1870CBC,4205 9567,NaN,NaN
4,6049020,6,6049,Espacios de Exhibición Patrimonial,Bartolomé J. Ronco 654,B7300XAA,434 811,museoazul@yahoo.com,NaN
...,...,...,...,...,...,...,...,...,...
1177,90098040,90,90098,Espacios de Exhibición Patrimonial,La Banda S/N,4000,421 685,NaN,NaN
1178,90105080,90,90105,Espacios de Exhibición Patrimonial,Centro Principal el Cadillal,NaN,NaN,NaN,NaN
1179,90105080,90,90105,Espacios de Exhibición Patrimonial,Avda. Pte. Roque Sáenz Peña 234,4103,461 4335,NaN,NaN
1180,90105070,90,90105,Espacios de Exhibición Patrimonial,Ernesto Padilla S/N,4101,422 2199,NaN,NaN


### cines

In [20]:
df_cines = get_response_and_create_csv(list_dicts_url[1])

In [21]:
df_cines.sample(5)

,index,cod_localidad,id_provincia,id_departamento,categoria,provincia,departamento,localidad,nombre,direccion,...,latitud,longitud,tipo_latitud_longitud,fuente,sector,pantallas,butacas,tipo_de_gestion,espacio_incaa,año_actualizacion
176,176,14021250,14,14021,Salas de cine,Córdoba,Colón,Salsipuedes,Dinosaurio Mall Salsipuedes,Av. Sarmiento 1007,...,-31.126828,-64.296023,Precisa,INCAA / SInCA,Cine,2,322,Privada,No,2022
207,207,38021060,38,38021,Salas de cine,Jujuy,Dr. Manuel Belgrano,San Salvador de Jujuy,Cine Auditorium,Independencia Esq. Italia S/N,...,-24.187252,-65.292851,Precisa,INCAA / SInCA,Cine,1,201,NaN,Si,2022
305,305,90084010,90,90084,Salas de cine,Tucumán,Capital,San Miguel de Tucumán,Atlas Vía 24,24 de Septiembre 757,...,-26.830169,-65.208742,Precisa,INCAA / SInCA,Cine,2,264,Privada,No,2022
35,35,6441030,6,6441,Salas de cine,Buenos Aires,La Plata,La Plata,Cinema 8,Calle 8 981,...,-34.916996,-57.949315,Precisa,INCAA / SInCA,Cine,3,1084,Privada,No,2022
219,219,46014010,46,46014,Salas de cine,La Rioja,Capital,La Rioja,Cinemacenter La Rioja,Av. Abel Bazan Y Bustos 710,...,-29.411525,-66.858150,Precisa,INCAA / SInCA,Cine,4,769,Privada,No,2022


In [22]:
df_cines.columns

Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'categoria', 'provincia', 'departamento', 'localidad', 'nombre',
       'direccion', 'piso', 'cp', 'web', 'latitud', 'longitud',
       'tipo_latitud_longitud', 'fuente', 'sector', 'pantallas', 'butacas',
       'tipo_de_gestion', 'espacio_incaa', 'año_actualizacion'],
      dtype='object')

In [23]:
new_columns = [i for i in norm_cols if i not in df_cines.columns]
new_columns.remove("número de teléfono")
new_columns.remove("mail")
new_columns

['categoría', 'domicilio', 'código postal']

In [24]:

old_columns = ['categoria',
 'direccion',
 'cp',]

In [25]:
df_cines=columns_rename(df_cines, old_columns, new_columns)
df_cines[new_columns]

{'categoria': 'categoría', 'direccion': 'domicilio', 'cp': 'código postal'}


,categoría,domicilio,código postal
0,Salas de cine,Calle 9 entre 29 Y 30,6660
1,Salas de cine,Santa Rosa De Lima 170,6405
2,Salas de cine,Robbio 840,6500
3,Salas de cine,Hipolito Yrigoyen 13200,1846
4,Salas de cine,Segui 699,1846
...,...,...,...
311,Salas de cine,Av. Nestor Kirchner(Ex Av. Roca) 3450,4000
312,Salas de cine,San Martin 251,4000
313,Salas de cine,Av. Mitre 817,4000
314,Salas de cine,Av. Aconquija 1336,4107


### Espacios

In [26]:
df_espacios = get_response_and_create_csv(list_dicts_url[2])

In [27]:
new_columns = [i for i in norm_cols if i not in df_espacios.columns]
new_columns

['categoría', 'código postal', 'número de teléfono']

In [28]:
df_espacios.columns

Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'observacion', 'categoria', 'subcategoria', 'provincia', 'departamento',
       'localidad', 'nombre', 'domicilio', 'piso', 'cp', 'cod_tel', 'telefono',
       'mail', 'web', 'informacion_adicional', 'latitud', 'longitud',
       'tipo_latitud_longitud', 'fuente', 'fecha_fundacion', 'nro_conabip',
       'anio_actualizacion'],
      dtype='object')

In [29]:
old_columns = ['categoria',
 'cp',
 'telefono',]
df_espacios=columns_rename(df_espacios, old_columns, new_columns)

{'categoria': 'categoría', 'cp': 'código postal', 'telefono': 'número de teléfono'}


In [30]:
df_espacios[new_columns]

,categoría,código postal,número de teléfono
0,Biblioteca Popular,6660,462234.0
1,Biblioteca Popular,6500,NaN
2,Biblioteca Popular,6500,426779.0
3,Biblioteca Popular,B6430CSL,430724.0
4,Biblioteca Popular,6441,432073.0
...,...,...,...
1897,Biblioteca Popular,4103,4619066.0
1898,Biblioteca Popular,4103,4461183.0
1899,Biblioteca Popular,4103,4610234.0
1900,Biblioteca Popular,4101,4370749.0


### Registros totales

In [31]:
list_df: List[pd.DataFrame]=[df_museo,df_cines,df_espacios]


In [32]:

list_colums = ["código postal", "número de teléfono", "mail", "web"]

In [33]:
df_all_culture_space=pd.concat([df_museo, df_cines, df_espacios],keys=["museos","cines","espacios"])

In [34]:
df_all_culture_space["one"]=1

#### por categoria, fuente y provincias

In [35]:
registros_by_category = df_all_culture_space.value_counts("categoría")
registros_by_category

categoría
Biblioteca Popular                    1902
Espacios de Exhibición Patrimonial    1182
Salas de cine                          316
Name: count, dtype: int64

In [36]:
registros_by_category.reset_index()

,categoría,count
0,Biblioteca Popular,1902
1,Espacios de Exhibición Patrimonial,1182
2,Salas de cine,316


In [37]:
registros_by_source = df_all_culture_space.value_counts("fuente")
registros_by_source

fuente
CONABIP                                                                1902
DNPyM                                                                  1048
INCAA / SInCA                                                           316
Municipalidad de Santa Fe - Red de Ciudades Creativas                    37
RCC- Córdoba                                                             31
Gobierno de la Provincia                                                 14
RCC                                                                      10
Área Gestión Cultural de la Municipalidad de Colonia Caroya               4
Red de Espacios Culturales Vicente López                                  4
Gobierno de la Provincia de Mendoza                                       4
Gobierno de la Provincia de Jujuy                                         4
Municipalidad de Río Gallegos - Red de Ciudades Creativas                 4
Gobierno de la Provincia de Chubut / SInCA 2013                           3
Gobie

In [38]:
registros_by_province = df_all_culture_space.groupby(["provincia","categoría"]).agg(
    count=("categoría","count") #*new_name_column = ("column_df map", "agg funtion")*
    )
registros_by_province.reset_index(inplace=True) # para que no quede la data like a groupby type, else like a dataframe
registros_by_province

,provincia,categoría,count
0,Buenos Aires,Biblioteca Popular,497
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,101
3,Catamarca,Biblioteca Popular,37
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
67,Tierra del Fuego,Salas de cine,3
68,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,14
69,Tucumán,Biblioteca Popular,38
70,Tucumán,Espacios de Exhibición Patrimonial,26


### registros del cine

In [39]:
# Cines por provincias

df_cines.value_counts("provincia")

provincia
Buenos Aires                       101
Ciudad Autónoma de Buenos Aires     37
Córdoba                             36
Santa Fe                            21
Mendoza                             17
Tucumán                             11
Entre Ríos                          11
Chubut                               8
Santa Cruz                           8
Salta                                7
Corrientes                           7
Neuquén                              7
La Rioja                             6
Río Negro                            6
La Pampa                             5
Misiones                             4
Jujuy                                4
San Juan                             4
Santiago del Estero                  4
San Luis                             3
Chaco                                3
Tierra del Fuego                     3
Catamarca                            2
Formosa                              1
Name: count, dtype: int64

In [40]:
# cantidad de pantallas
df_cines.value_counts("pantallas")


pantallas
1     152
2      50
3      24
6      17
4      16
5      14
8      11
10      9
7       8
9       6
12      3
14      2
16      2
13      1
17      1
Name: count, dtype: int64

In [41]:
# cantidad de butacas
df_cines.value_counts("butacas")

butacas
350     7
300     7
400     5
220     4
500     4
       ..
419     1
430     1
431     1
432     1
4214    1
Name: count, Length: 257, dtype: int64

In [42]:
# cantidad de espacios incaa
df_cines.value_counts("espacio_incaa")

espacio_incaa
No    266
Si     50
Name: count, dtype: int64

In [43]:
df_cines['espacio_incaa']

0      No
1      No
2      No
3      No
4      No
       ..
311    No
312    Si
313    No
314    No
315    No
Name: espacio_incaa, Length: 316, dtype: object

In [44]:
df_cines.columns

Index(['index', 'cod_localidad', 'id_provincia', 'id_departamento',
       'categoría', 'provincia', 'departamento', 'localidad', 'nombre',
       'domicilio', 'piso', 'código postal', 'web', 'latitud', 'longitud',
       'tipo_latitud_longitud', 'fuente', 'sector', 'pantallas', 'butacas',
       'tipo_de_gestion', 'espacio_incaa', 'año_actualizacion'],
      dtype='object')

### registros totales por provincia

In [45]:
count_si= lambda x: sum(x=="Si")
df_cines.groupby("provincia", as_index=False).agg(
    cines = pd.NamedAgg(column="provincia", aggfunc="count"),
    pantallas = pd.NamedAgg(column="pantallas", aggfunc="sum"),
    butacas = pd.NamedAgg(column="butacas", aggfunc="sum"),
    espacio_incaa = pd.NamedAgg(column="espacio_incaa", aggfunc=count_si),
     )
#.agg({"pantallas":"sum", "butacas":"sum", "espacio_incaa":count_si})


,provincia,cines,pantallas,butacas,espacio_incaa
0,Buenos Aires,101,362,92818,18
1,Catamarca,2,10,1657,0
2,Chaco,3,13,2302,0
3,Chubut,8,9,2602,3
4,Ciudad Autónoma de Buenos Aires,37,150,27940,1
5,Corrientes,7,16,3170,0
6,Córdoba,36,110,22334,2
7,Entre Ríos,11,14,3842,1
8,Formosa,1,3,619,0
9,Jujuy,4,5,2277,2


## SQL

In [46]:
# from dataclasses import dataclass
# from typing import Dict, Any, Iterable
# from pandas import DataFrame
# from sqlalchemy import create_engine, inspect
# import urllib

# @dataclass(frozen=True)
# class ConnectionSettings:
#     """Connection Settings."""
#     server: str
#     database: str
#     username: str
#     password: str
#     driver: str = '{ODBC Driver 18 for SQL Server}'
#     timeout: int = 30

# class AzureDbConnection:
#     """
#     Azure SQL database connection.
#     """
#     def __init__(self, conn_settings: ConnectionSettings, echo: bool = False) -> None:
#         conn_params = urllib.parse.quote_plus(
#             'Driver=%s;' % conn_settings.driver +
#             'Server=tcp:%s.database.windows.net,1433;' % conn_settings.server +
#             'Database=%s;' % conn_settings.database +
#             'Uid=%s;' % conn_settings.username +
#             'Pwd=%s;' % conn_settings.password +
#             'Encrypt=yes;' +
#             'TrustServerCertificate=no;' +
#             'Connection Timeout=%s;' % conn_settings.timeout
#         )
#         conn_string = f'mssql+pyodbc:///?odbc_connect={conn_params}'

#         self.db = create_engine(conn_string, echo=echo)

#     def connect(self) -> None:
#         """Estimate connection."""
#         self.conn = self.db.connect()

#     def get_tables(self) -> Iterable[str]:
#         """Get list of tables."""
#         inspector = inspect(self.db)
#         return [t for t in inspector.get_table_names()]

#     def dispose(self) -> None:
#         """Dispose opened connections."""
#         self.conn.close()
#         self.db.dispose()

In [47]:
# cfg= ConnectionSettings
# cfg.database = "GaliciaFake"
# cfg.server ="localhost\SQLEXPRESS"
# cfg.username = ""
# cfg.password = ""
# cfg.driver = ""

In [48]:
# conn=AzureDbConnection(cfg).connect()

In [4]:
import sqlite3
import pandas as pd
con_sqlite = sqlite3.connect("cultura.db")

In [5]:
norm_cols= [
    "cod_localidad",
    "id_provincia",
    "id_departamento",
    "categoría",
    "provincia",
    "localidad",
    "nombre",
    "domicilio",
    "código postal",
    "número de teléfono",
    "mail",
    "web"
]

columns_for_sql= "["+"], [".join(norm_cols)+"]"

def run_sql(con: sqlite3.Connection ,script:str):
    try:
        #no se como hacer para que el index se auto complete
        con.execute(script)
        con.commit()
    except Exception as e:
        print(f"no funciono, tu error es: {e}")

In [ ]:
for num,df in enumerate(list_df):
    df.to_sql(list_dicts_url[num]["name"], con_sqlite, index= False, if_exists="replace")

### Select, select distinct, select as

In [ ]:
sql_script="SELECT categoría, provincia FROM museo" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT count(*) as papaya FROM (select distinct provincia from museo)" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT distinct provincia as papaya FROM museo" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT count(distinct provincia), count(distinct categoría) FROM museo" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT distinct provincia FROM museo" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT distinct provincia FROM cines" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas FROM cines where provincia like 'ca%'" # SELECT (COLUMN) FROM (TABLE NAME)
pd.read_sql(sql=sql_script, con= con_sqlite)


,provincia,butacas
0,Catamarca,914
1,Catamarca,743


### Select where, =, <, >, <=, >=, <>

In [4]:
sql_script="SELECT provincia FROM museo where provincia = 'Catamarca'" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT count(provincia) FROM museo where provincia = 'Catamarca'" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT distinct id_provincia FROM cines" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia FROM cines where id_provincia = 6" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia FROM cines where id_provincia != 6" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia FROM cines where id_provincia <> 6" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia FROM cines where provincia like 'co%'" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia FROM cines where id_provincia between 6 and 20" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia FROM cines where id_provincia in (6,18,3)" # SELECT (COLUMNs) FROM (TABLE NAME)
pd.read_sql(sql=sql_script, con= con_sqlite)


,provincia,id_provincia
0,Buenos Aires,6
1,Buenos Aires,6
2,Buenos Aires,6
3,Buenos Aires,6
4,Buenos Aires,6
...,...,...
103,Corrientes,18
104,Corrientes,18
105,Corrientes,18
106,Corrientes,18


### select order by, asc, desc

In [ ]:
sql_script="SELECT provincia, id_provincia from cines order by id_provincia asc" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia from cines order by provincia asc" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia from cines order by id_provincia desc" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia from cines order by provincia desc" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, id_provincia from cines order by provincia asc, id_provincia desc" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas from cines order by provincia asc, butacas desc" # SELECT (COLUMNs) FROM (TABLE NAME)
pd.read_sql(sql=sql_script, con= con_sqlite)

,provincia,butacas
0,Buenos Aires,4214
1,Buenos Aires,3890
2,Buenos Aires,3485
3,Buenos Aires,3427
4,Buenos Aires,2675
...,...,...
311,Tucumán,183
312,Tucumán,80
313,Tucumán,50
314,Tucumán,50


### select where and, or, not

In [ ]:
sql_script="SELECT provincia, butacas from cines where provincia like 't%' and butacas > 300" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas from cines where provincia like 't%' and butacas > 300 order by butacas" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas from cines where provincia like 't%' and butacas in (300, 750)" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas from cines where provincia like 't%' and (butacas < 700 or butacas < 1000 )" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas from cines where not provincia like 't%' and (butacas < 700 or butacas < 1000 )" # SELECT (COLUMNs) FROM (TABLE NAME)
pd.read_sql(sql=sql_script, con= con_sqlite)

,provincia,butacas
0,Buenos Aires,500
1,Buenos Aires,410
2,Buenos Aires,165
3,Buenos Aires,710
4,Buenos Aires,160
...,...,...
227,Santa Fe,380
228,Santa Fe,249
229,Santiago del Estero,152
230,Santiago del Estero,822


### INSERT INTO

In [ ]:
sql_script="INSERT into cines (provincia, butacas) values ('Guanare', 100)" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="INSERT into cines (provincia, butacas) values ('Guanare', 100), ('Guarenas', 150), ('Guatire', 50)" # SELECT (COLUMNs) FROM (TABLE NAME)
run_sql(con_sqlite,sql_script)

### WHERE IS NULL, NOT NULL

In [ ]:
sql_script="SELECT provincia, butacas from cines where butacas is null" # SELECT (COLUMNs) FROM (TABLE NAME)
sql_script="SELECT provincia, butacas from cines where butacas is not null" # SELECT (COLUMNs) FROM (TABLE NAME)
pd.read_sql(sql=sql_script, con= con_sqlite)

,provincia,butacas
0,Buenos Aires,500
1,Buenos Aires,410
2,Buenos Aires,165
3,Buenos Aires,2505
4,Buenos Aires,710
...,...,...
315,Tucumán,1295
316,Guanare,100
317,Guanare,100
318,Guarenas,150


### UPDATE, SET

In [ ]:
#Note: Be careful when updating records in a table! Notice the WHERE clause in the UPDATE statement. The WHERE clause specifies which record(s) that should be updated. If you omit the WHERE clause, all records in the table will be updated!

sql_script="UPDATE cines set categoría = 'Salas de cine' where provincia = 'Guanare'" # SELECT (COLUMNs) FROM (TABLE NAME)

run_sql(con_sqlite,sql_script)

### DELETE FROM

In [50]:
sql_script="DELETE from cines where provincia = 'Guarenas' or provincia = 'Guatire' " # SELECT (COLUMNs) FROM (TABLE NAME)

run_sql(con_sqlite,sql_script)

In [ ]:
# It is possible to delete all rows in a table without deleting the table. This means that the table structure, attributes, and indexes will be intact

sql_script="delete from table_name"

In [ ]:
# To delete the table completely, use the DROP TABLE statement

sql_script= "DROP TABLE table_name;"


### SELECT top, limit

In [72]:
sql_script= "select * from cines limit 5;"
sql_script= "select * from cines where butacas > 700 limit 5;"
sql_script= "select provincia, butacas from cines where butacas > 700 order by provincia desc limit 5;"

pd.read_sql(sql_script,con_sqlite)

,provincia,butacas
0,Tucumán,797
1,Tucumán,1051
2,Tucumán,750
3,Tucumán,1295
4,Santiago del Estero,822


### min, max, count, avg, sum

In [84]:
sql_script= "select min(butacas) from cines where butacas > 700;"
sql_script= "select max(butacas) from cines where butacas > 700;"
sql_script= "select max(butacas) as 'upper butacas' from cines where butacas > 700;"
sql_script= "select count(provincia) from cines;"
sql_script= "select count(provincia) from cines where butacas > 1000;"
sql_script= "select count(categoría) from cines where butacas > 1000;"
# If you specify a column instead of (*), NULL values will not be counted.
sql_script= "select count(categoría) from cines where provincia = 'caracas';"

#You can ignore duplicates by using the DISTINCT keyword in the COUNT function.

# If DISTINCT is specified, rows with the same value for the specified column will be counted as one.
sql_script= "select count(distinct provincia) from cines where butacas > 1000;"

pd.read_sql(sql_script,con_sqlite)

,count(distinct provincia)
0,13


In [92]:
sql_script= "select sum(butacas) from cines where provincia like 'gua%';"
sql_script= "select sum((butacas / 10) + 1) from cines where provincia like 'gua%';"

pd.read_sql(sql_script,con_sqlite)

,sum((butacas / 10) + 1)
0,22


In [99]:
sql_script= "select avg(butacas) from cines where provincia like 'gua%';"
sql_script= "select provincia from cines where butacas < (select  avg(butacas) from cines);"

pd.read_sql(sql_script,con_sqlite)

,provincia
0,Buenos Aires
1,Buenos Aires
2,Buenos Aires
3,Buenos Aires
4,Buenos Aires
...,...
209,Tucumán
210,Tucumán
211,Tucumán
212,Guanare


### LIKE, NOT LIKE

In [115]:
# There are two wildcards often used in conjunction with the LIKE operator:

#  The percent sign % represents zero, one, or multiple characters
#  The underscore sign _ represents one, single character

sql_script= "select provincia from cines where provincia like 'g__n%';"
sql_script= "select provincia from cines where provincia like '%g%';"
sql_script= "select provincia from cines where provincia like '%';"

pd.read_sql(sql_script,con_sqlite)

,provincia
0,Buenos Aires
1,Buenos Aires
2,Buenos Aires
3,Buenos Aires
4,Buenos Aires
...,...
316,caracas
317,caracas
318,caracas
319,Guanare


### IN, NOT IN

In [140]:

sql_script= "select provincia from museo where provincia in (select provincia from cines where butacas between 0 and 20);"
sql_script= "select provincia from museo where provincia in (select provincia from cines where butacas between 0 and 20);"

pd.read_sql(sql_script,con_sqlite)

,provincia
0,Buenos Aires
1,Buenos Aires
2,Buenos Aires
3,Buenos Aires
4,Buenos Aires
...,...
147,Buenos Aires
148,Buenos Aires
149,Buenos Aires
150,Buenos Aires


### BETWEEN, NOT BETWEEN

In [153]:
# The BETWEEN operator selects values within a given range. The values can be numbers, text, or dates.

# The BETWEEN operator is inclusive: begin and end values are included. 

sql_script= "select provincia from cines where departamento between 'Capital' and 'Yerba Buena';"
sql_script= "select año_actualizacion from cines where año_actualizacion between #01/01/2000# and #01/01/2017#;"
sql_script= "select año_actualizacion from cines where año_actualizacion between '01-01-2000' and '01-01-2017';"
sql_script= "select provincia from cines where departamento between 'Capital' and 'Yerba Buena' order by departamento;"

pd.read_sql(sql_script,con_sqlite)

,provincia
0,Catamarca
1,Catamarca
2,Córdoba
3,Córdoba
4,Córdoba
...,...
288,Buenos Aires
289,Entre Ríos
290,Tucumán
291,Tucumán


### Alias

Aliases can be useful when:

There are more than one table involved in a query
Functions are used in the query
Column names are big or not very readable
Two or more columns are combined together

In [18]:
sql_script= "select subcategoria, (categoría || ', ' || provincia) as adress from museo;" # work as oracle db
sql_script= "select subcategoria, (categoría || ', ' || provincia) as adress from museo as museos;" # work as oracle db
# short sql script
sql_script= "select m.localidad as [m.localidad], m.nombre as [m.nombre], c.localidad as [c.localidad], c.nombre as [c.nombre] from museo as m, cines as c where m.cod_localidad = c.cod_localidad;" # work as oracle db

pd.read_sql(sql_script,con_sqlite)

,m.localidad,m.nombre,c.localidad,c.nombre
0,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,9 de julio,Tu Cine 9 de Julio
1,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,Avellaneda,Cinemark Alto Avellaneda
2,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,Avellaneda,Cinepolis Avellaneda
3,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,Avellaneda,Cinemark Alto Avellaneda
4,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,Avellaneda,Cinepolis Avellaneda
...,...,...,...,...
6747,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,San Miguel de Tucumán,Cine Popular San Miguel de Tucumán
6748,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,San Miguel de Tucumán,Cinemacenter Tucuman
6749,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,San Miguel de Tucumán,Espacio Incaa Sala Orestes Caviglia
6750,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,San Miguel de Tucumán,Microcine Orion


### joins

A **JOIN** clause is used to combine rows from two or more tables, based on a related column between them.

In [6]:
sql_script= "select m.localidad as [m.localidad], m.nombre as [m.nombre], c.nombre as [c.nombre] from museo as m, cines as c inner join museo on m.cod_localidad = c.cod_localidad;" # work as oracle db
sql_script= f"select m.localidad as [m.localidad], m.nombre as [m.nombre], c.nombre as [c.nombre] from museo as m, cines as c inner join museo on m.cod_localidad = c.cod_localidad;" # work as oracle db

pd.read_sql(sql_script,con_sqlite)

,m.localidad,m.nombre,c.nombre
0,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Tu Cine 9 de Julio
1,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Tu Cine 9 de Julio
2,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Tu Cine 9 de Julio
3,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Tu Cine 9 de Julio
4,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Tu Cine 9 de Julio
...,...,...,...
7980859,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,Cine Popular San Miguel de Tucumán
7980860,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,Cinemacenter Tucuman
7980861,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,Espacio Incaa Sala Orestes Caviglia
7980862,San Miguel de Tucumán,Museo Del Ejército - Guarnición Militar Tucumán,Microcine Orion


Here are the different types of the JOINs in SQL:

- (INNER) JOIN: Returns records that have matching values in both tables
- LEFT (OUTER) JOIN: Returns all records from the left table, and the matched records from the right table
- RIGHT (OUTER) JOIN: Returns all records from the right table, and the matched records from the left table
- FULL (OUTER) JOIN: Returns all records when there is a match in either left or right table

Note: The INNER JOIN keyword returns only rows with a match in both tables. Which means that if you have a product with no CategoryID, or with a CategoryID that is not present in the Categories table, that record would not be returned in the result.

It is a good practice to include the table name when specifying columns in the SQL statement.

JOIN and INNER JOIN will return the same result.

INNER is the default join type for JOIN, so when you write JOIN the parser actually writes INNER JOIN.

The LEFT JOIN keyword returns all records from the left table (table1), and the matching records from the right table (table2). The result is 0 records from the right side, if there is no match.

Note: In some databases LEFT JOIN is called LEFT OUTER JOIN.

Note: The LEFT JOIN keyword returns all records from the left table (Customers), even if there are no matches in the right table (Orders).

The RIGHT JOIN keyword returns all records from the right table (table2), and the matching records from the left table (table1). The result is 0 records from the left side, if there is no match.

Note: In some databases RIGHT JOIN is called RIGHT OUTER JOIN.

Note: The RIGHT JOIN keyword returns all records from the right table (Employees), even if there are no matches in the left table (Orders).

The FULL OUTER JOIN keyword returns all records when there is a match in left (table1) or right (table2) table records.

Tip: FULL OUTER JOIN and FULL JOIN are the same.

Note: FULL OUTER JOIN can potentially return very large result-sets!

Note: The FULL OUTER JOIN keyword returns all matching records from both tables whether the other table matches or not. So, if there are rows in "Customers" that do not have matches in "Orders", or if there are rows in "Orders" that do not have matches in "Customers", those rows will be listed as well.

A self join is a regular join, but the table is joined with itself.

SELECT column_name(s)
FROM table1 T1, table1 T2
WHERE condition;

The following SQL statement matches customers that are from the same city:

SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
FROM Customers A, Customers B
WHERE A.CustomerID <> B.CustomerID
AND A.City = B.City
ORDER BY A.City;

### UNION

The UNION operator is used to combine the result-set of two or more SELECT statements.

- Every SELECT statement within UNION must have the same number of columns
- The columns must also have similar data types
- The columns in every SELECT statement must also be in the same order

''' 
SELECT column_name(s) FROM table1

UNION

SELECT column_name(s) FROM table2;
'''

The UNION operator selects only distinct values by default. To allow duplicate values, use UNION ALL

Note: The column names in the result-set are usually equal to the column names in the first SELECT statement.

The UNION operator selects only distinct values by default. To allow duplicate values, use UNION ALL

Note: The column names in the result-set are usually equal to the column names in the first SELECT statement.

In [20]:
sql_script=f"SELECT {columns_for_sql} FROM museo union SELECT {columns_for_sql} FROM espacios"
sql_script=f"SELECT {columns_for_sql} FROM museo limit 5" # SELECT (COLUMN) FROM (TABLE NAME)
sql_script=f"SELECT distinct cod_localidad FROM museo" # SELECT (COLUMN) FROM (TABLE NAME)
sql_script=f"SELECT {columns_for_sql} FROM museo union all SELECT {columns_for_sql} FROM espacios limit 5" # seria interesante un modo de obtener lo parecidos
pd.read_sql(sql=sql_script, con= con_sqlite)

,cod_localidad,id_provincia,id_departamento,categoría,provincia,localidad,nombre,domicilio,código postal,número de teléfono,mail,web
0,6588100,6,6588,Espacios de Exhibición Patrimonial,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Libertad 1191,B6500EVL,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar
1,6077010,6,6077,Espacios de Exhibición Patrimonial,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,Gerardo Risso y España,B2740FMJ,452 931,museoarrecifes@hotmail.com,None
2,6035010,6,6035,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,Calle 11 (Av. San Martín) Nº 830,S3561AKT,481 200,None,None
3,6035010,6,6035,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,"Sarmiento 101, 1 º Piso",B1870CBC,4205 9567,None,None
4,6049020,6,6049,Espacios de Exhibición Patrimonial,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,Bartolomé J. Ronco 654,B7300XAA,434 811,museoazul@yahoo.com,None


### The GROUP BY

The GROUP BY statement groups rows that have the same values into summary rows, like "find the number of customers in each country".

The GROUP BY statement is often used with aggregate functions (COUNT(), MAX(), MIN(), SUM(), AVG()) to group the result-set by one or more columns.

SELECT column_name(s)

FROM table_name

WHERE condition

GROUP BY column_name(s)

ORDER BY column_name(s);

In [27]:
sql_script=f"SELECT {columns_for_sql} FROM museo group by provincia" # seria interesante un modo de obtener lo parecidos
sql_script=f"SELECT count(provincia), provincia FROM museo group by provincia" # seria interesante un modo de obtener lo parecidos
sql_script=f"SELECT count(provincia), provincia, localidad FROM museo group by localidad" # seria interesante un modo de obtener lo parecidos
sql_script=f"""
SELECT 
count(categoría), provincia
FROM museo
group by provincia
order by count(categoría) desc
""" # seria interesante un modo de obtener lo parecidos
pd.read_sql(sql=sql_script, con= con_sqlite)

,count(categoría),provincia
0,162,Santa Fe
1,152,Buenos Aires
2,132,Ciudad Autónoma de Buenos Aires
3,126,Córdoba
4,63,Mendoza
5,57,Entre Ríos
6,47,Jujuy
7,45,La Pampa
8,45,Chubut
9,39,Neuquén


### HAVING

The HAVING clause was added to SQL because the WHERE keyword cannot be used with aggregate functions.

SELECT column_name(s)

FROM table_name

WHERE condition

GROUP BY column_name(s)

HAVING condition

ORDER BY column_name(s);

In [29]:
sql_script=f"""
SELECT 
count(categoría), provincia
FROM museo
group by provincia
having count(categoría) between 5 and 10
""" # seria interesante un modo de obtener lo parecidos
pd.read_sql(sql=sql_script, con= con_sqlite)

,count(categoría),provincia
0,7,Formosa
1,10,Santiago del Estero


### EXISTS

The SQL EXISTS Operator
- The EXISTS operator is used to test for the existence of any record in a subquery.

- The EXISTS operator returns TRUE if the subquery returns one or more records.

SELECT column_name(s)

FROM table_name

WHERE EXISTS

(SELECT column_name FROM table_name WHERE condition);

In [25]:
sql_script=f"""
SELECT *
FROM museo
where exists (select provincia from museo where id_provincia > 90 )
""" 
# si lo del parentesis cumple genera muestra la consulta
sql_script=f"""
SELECT *
FROM museo
where exists (select provincia from cines where butacas > 4000 )
""" 
pd.read_sql(sql=sql_script, con= con_sqlite)

,index,cod_localidad,id_provincia,id_departamento,Observaciones,categoría,subcategoria,provincia,localidad,nombre,...,mail,web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
0,0,6588100,6,6588,None,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,...,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,-35.441762,-60.887598,Localización precisa,None,DNPyM,Municipal,1920.0,2017
1,1,6077010,6,6077,None,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,...,museoarrecifes@hotmail.com,None,-34.066970,-60.102555,Localización precisa,None,DNPyM,Municipal,1972.0,2017
2,2,6035010,6,6035,None,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,...,None,None,-29.118024,-59.654543,Localización precisa,None,DNPyM,Municipal,NaN,2017
3,3,6035010,6,6035,None,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,...,None,None,-34.662110,-58.363270,Localización precisa,None,DNPyM,Municipal,1956.0,2017
4,4,6049020,6,6049,None,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,...,museoazul@yahoo.com,None,-36.781824,-59.858636,Localización precisa,None,DNPyM,Privada,1949.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,1177,90098040,90,90098,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí del Valle,Museo Histórico Y Arqueológico Jesuítico La Banda,...,None,None,-26.852880,-65.712622,Localización precisa,None,DNPyM,None,NaN,2017
1178,1178,90105080,90,90105,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Mac-Museo Arqueológico El Cadillal,...,None,None,-26.683331,-65.266670,Localización precisa,None,DNPyM,Provincial,NaN,2017
1179,1179,90105080,90,90105,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Museo Ferroviario De Los Talleres De Tafí Viejo,...,None,None,-26.740733,-65.263231,Localización precisa,None,DNPyM,Privada,1909.0,2017
1180,1180,90105070,90,90105,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Villa El Cadillal,Museo Arqueológico Dr. Ernesto Padilla,...,None,None,-21.937642,-66.052043,Localización precisa,None,DNPyM,Provincial,NaN,2017


### ANY and ALL operators

The SQL ANY and ALL Operators
The ANY and ALL operators allow you to perform a comparison between a single column value and a range of other values.

The SQL ANY Operator
The ANY operator:

returns a boolean value as a result
returns TRUE if ANY of the subquery values meet the condition
ANY means that the condition will be true if the operation is true for any of the values in the range.

SELECT column_name(s)

FROM table_name

WHERE column_name operator ANY

  (SELECT column_name

  FROM table_name
  
  WHERE condition);

Note: The operator must be a standard comparison operator (=, <>, !=, >, >=, <, or <=).

The ALL operator:

returns a boolean value as a result
returns TRUE if ALL of the subquery values meet the condition
is used with SELECT, WHERE and HAVING statements
ALL means that the condition will be true only if the operation is true for all values in the range. 

With select

SELECT ALL column_name(s)
FROM table_name
WHERE condition;

With where or having  

SELECT column_name(s)
FROM table_name
WHERE column_name operator ALL
  (SELECT column_name
  FROM table_name
  WHERE condition);



In [30]:
### SQLite dont use ANY or ALL operator
# sql_script=f"""
# SELECT *
# FROM museo
# where provincia operator all (select provincia from museo where id_provincia > 90 )
# """
# sql_script=f"""
# SELECT *
# FROM museo
# where provincia operator any (select provincia from museo where id_provincia > 90 )
# """
# pd.read_sql(sql=sql_script, con= con_sqlite)


### SELECT INTO

The SELECT INTO statement copies data from one table into a new table.

SELECT *

INTO newtable [IN externaldb]

FROM oldtable

WHERE condition;

The new table will be created with the column-names and types as defined in the old table. You can create new column names using the AS clause.

Tip: SELECT INTO can also be used to create a new, empty table using the schema of another. Just add a WHERE clause that causes the query to return no data:

SELECT * INTO newtable
FROM oldtable
WHERE 1 = 0;

In [39]:
sql_script=f"""
select * from id_num
"""
pd.read_sql(sql=sql_script, con= con_sqlite)


,index,cod_localidad,id_provincia,id_departamento,Observaciones,categoría,subcategoria,provincia,localidad,nombre,...,mail,web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
0,534,30028010,30,30028,None,Espacios de Exhibición Patrimonial,Museos,Entre Ríos,Chajari,Museo Municipal Ivy Mara Ey - La Tierra Sin Mal,...,None,None,-30.756532,-57.984198,Localización precisa,None,DNPyM,Municipal,NaN,2017
1,535,30028010,30,30028,None,Espacios de Exhibición Patrimonial,Museos,Entre Ríos,Chajari,Museo Regional Camila Quiroga,...,museocamilaquiroga@yahoo.com,None,-30.746825,-57.978500,Localización precisa,None,DNPyM,Municipal,1982.0,2017
2,536,30008020,30,30008,None,Espacios de Exhibición Patrimonial,Museos,Entre Ríos,Colón,Museo Histórico Museóloga María Angélica González,...,None,None,-32.221483,-58.136118,Localización precisa,None,DNPyM,Privada,1971.0,2017
3,537,30008020,30,30008,None,Espacios de Exhibición Patrimonial,Museos,Entre Ríos,Colón,Museo Provincial Molino Forclaz,...,None,None,-32.225643,-58.141999,Localización precisa,None,DNPyM,Provincial,2015.0,2017
4,538,30098040,30,30098,None,Espacios de Exhibición Patrimonial,Museos,Entre Ríos,Concepcion del Uruguay,Museo Histórico Evocativo Del Colegio Superior...,...,museo@elhistorico.com.ar,None,-32.484417,-58.233049,Localización precisa,None,DNPyM,Provincial,NaN,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,1177,90098040,90,90098,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí del Valle,Museo Histórico Y Arqueológico Jesuítico La Banda,...,None,None,-26.852880,-65.712622,Localización precisa,None,DNPyM,None,NaN,2017
1542,1178,90105080,90,90105,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Mac-Museo Arqueológico El Cadillal,...,None,None,-26.683331,-65.266670,Localización precisa,None,DNPyM,Provincial,NaN,2017
1543,1179,90105080,90,90105,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Museo Ferroviario De Los Talleres De Tafí Viejo,...,None,None,-26.740733,-65.263231,Localización precisa,None,DNPyM,Privada,1909.0,2017
1544,1180,90105070,90,90105,None,Espacios de Exhibición Patrimonial,Museos,Tucumán,Villa El Cadillal,Museo Arqueológico Dr. Ernesto Padilla,...,None,None,-21.937642,-66.052043,Localización precisa,None,DNPyM,Provincial,NaN,2017


In [36]:
# No tiene select into pero si lo puede aplicar con create

sql_script=f"""
CREATE TABLE id_num 
AS SELECT * FROM museo where id_provincia >= 30
"""
# run_sql(script=sql_script, con= con_sqlite)
pd.read_sql(sql=sql_script, con= con_sqlite) ## si lo ejecuta pero no tiene nada para mostrar


TypeError: 'NoneType' object is not iterable

### INSERT INTO SELECT

The INSERT INTO SELECT statement copies data from one table and inserts it into another table.

The INSERT INTO SELECT statement requires that the data types in source and target tables match.

Note: The existing records in the target table are unaffected.

INSERT INTO table2
SELECT * FROM table1
WHERE condition;


In [38]:
sql_script=f"""
INSERT INTO id_num
SELECT * FROM museo
WHERE id_provincia >= 10;
"""
run_sql(script=sql_script, con= con_sqlite)

### CASE

The CASE expression goes through conditions and returns a value when the first condition is met (like an if-then-else statement). So, once a condition is true, it will stop reading and return the result. If no conditions are true, it returns the value in the ELSE clause.

If there is no ELSE part and no conditions are true, it returns NULL.

CASE

    WHEN condition1 THEN result1

    WHEN condition2 THEN result2

    WHEN conditionN THEN resultN

    ELSE result
    
END;

In [1]:
sql_script=f"""
SELECT provincia, id_provincia,
CASE
    WHEN id_provincia > 30 THEN 'The quantity is greater than 30'
    WHEN id_provincia = 30 THEN 'The quantity is 30'
    ELSE 'The quantity is under 30'
END AS QuantityText
FROM museo;
"""
# run_sql(script=sql_script, con= con_sqlite)
pd.read_sql(sql=sql_script, con= con_sqlite) ## si lo ejecuta pero no tiene nada para mostrar

NameError: name 'pd' is not defined

### IFNULL(), ISNULL(), COALESCE(), NVL()

Look at the following "Products" table.

Suppose that the "web" column is optional, and may contain NULL values.

Look at the following SELECT statement:

SELECT ProductName, UnitPrice * (UnitsInStock + UnitsOnOrder)
FROM Products;

In [44]:

sql_script=f"""
SELECT provincia, mail * (id_provincia + IFNULL(web, 0))
FROM museo;
"""

sql_script=f"""
SELECT provincia, mail * (id_provincia + COALESCE(web, 0))
FROM museo;
"""
# run_sql(script=sql_script, con= con_sqlite)

pd.read_sql(sql=sql_script, con= con_sqlite)

,provincia,"mail * (id_provincia + COALESCE(web, 0))"
0,Buenos Aires,0.0
1,Buenos Aires,0.0
2,Buenos Aires,NaN
3,Buenos Aires,NaN
4,Buenos Aires,0.0
...,...,...
1177,Tucumán,NaN
1178,Tucumán,NaN
1179,Tucumán,NaN
1180,Tucumán,NaN


### What is a Stored Procedure?

A stored procedure is a prepared SQL code that you can save, so the code can be reused over and over again.

So if you have an SQL query that you write over and over again, save it as a stored procedure, and then just call it to execute it.

You can also pass parameters to a stored procedure, so that the stored procedure can act based on the parameter value(s) that is passed.

----------------------------------

CREATE PROCEDURE procedure_name

AS

sql_statement

GO;

-----------------------------

EXEC procedure_name;

In [4]:
"sqlite don't have create procedure"

"sqlite don't have create procedure"

### CREATE DATABASE

The CREATE DATABASE statement is used to create a new SQL database.

Tip: Make sure you have admin privilege before creating any database. Once a database is created, you can check it in the list of databases with the following SQL command: SHOW DATABASES;

In [16]:
sql_script=f"""
ATTACH DATABASE 'movies.db' AS Movies;
"""
# crea la base de datos y conecta con la misma

sql_script = """
PRAGMA database_list;
"""
sql_script = """
SELECT name FROM sqlite_master WHERE type='table';
"""

sql_script=f"""
DETACH DATABASE 'movies.db';
"""
# Desconecta con la base de datos

run_sql(con= con_sqlite, script= sql_script)

no funciono, tu error es: no such database: movies.db


### DROP DATABASE

The DROP DATABASE statement is used to drop an existing SQL database.

Note: Be careful before dropping a database. Deleting a database will result in loss of complete information stored in the database!

Tip: Before dropping a database, always check if the database exists using the SHOW DATABASES statement.

In [17]:
### DROP DATABASE don't exist in sqlite, because the db is a simple archive

### BACKUP DATABASE

The BACKUP DATABASE statement is used in SQL Server to create a full back up of an existing SQL database.

But sqlite don't have this command because the db is only a file.

### CREATE TABLE

The CREATE TABLE statement is used to create a new table in a database.

CREATE TABLE table_name (

    column1 datatype,
    
    column2 datatype,
    
    column3 datatype,
    
   ....
);

In [3]:
sql_script=f"""
create table two_table (
contact_id INTEGER PRIMARY KEY,
name text,
age text,
);
"""

run_sql(con= con_sqlite, script= sql_script)

no funciono, tu error es: near ")": syntax error


### ALTER TABLE

The ALTER TABLE statement is used to add, delete, or modify columns in an existing table.

The ALTER TABLE statement is also used to add and drop various constraints on an existing table.

In [11]:
sql_script=f"""ALTER TABLE museo
drop column column_name;""" ### solo elimina
sql_script=f"""ALTER TABLE museo
ADD column_name real;""" ### solo crea
sql_script=f"""ALTER TABLE museo
rename column column_name to column_name2;""" ### solo elimina



run_sql(con= con_sqlite, script= sql_script)

no funciono, tu error es: near "modify": syntax error


In [ ]:
## sqlite can't alter the type for a column but you can apply this method

def change_column_data_type(db_name, table_name, old_column_name, new_column_name, new_column_type):
    con = sqlite3.connect(db_name)
    cursor = con.cursor()

    # Step 1: Create a new table with the new column data type
    cursor.execute(f"""
    CREATE TABLE new_table AS SELECT
    CAST({old_column_name} AS {new_column_type}) AS {new_column_name},
    *
    FROM {table_name}
    """)

    # Step 2: Drop the old table
    cursor.execute(f"DROP TABLE {table_name}")

    # Step 3: Rename the new table to the old table name
    cursor.execute(f"ALTER TABLE new_table RENAME TO {table_name}")

    con.commit()
    con.close()

change_column_data_type('your_database_name', 'your_table_name', 'old_column_name', 'new_column_name', 'new_column_type')



### SQL Constraints

SQL constraints are used to specify rules for the data in a table.

Constraints are used to limit the type of data that can go into a table. This ensures the accuracy and reliability of the data in the table. If there is any violation between the constraint and the data action, the action is aborted.

Constraints can be column level or table level. Column level constraints apply to a column, and table level constraints apply to the whole table.

The following constraints are commonly used in SQL:

- NOT NULL - Ensures that a column cannot have a - NULL value
- UNIQUE - Ensures that all values in a column are different
- PRIMARY KEY - A combination of a NOT NULL and UNIQUE. Uniquely identifies each row in a table
- FOREIGN KEY - Prevents actions that would destroy links between tables
- CHECK - Ensures that the values in a column satisfies a specific condition
- DEFAULT - Sets a default value for a column if no value is specified
- CREATE INDEX - Used to create and retrieve data from the database very quickly

In [6]:
sql_script=f"""
CREATE TABLE Employees (
    ID INTEGER PRIMARY KEY,
    NAME TEXT NOT NULL,
    AGE INT NOT NULL CHECK(AGE >= 18),
    ADDRESS CHAR(50),
    SALARY REAL NOT NULL,
    UNIQUE(ID, NAME)
);"""
run_sql(con= con_sqlite, script= sql_script)


sqlite only have:

1. `NOT NULL`: This constraint ensures that a column cannot have a NULL value.

2. `UNIQUE`: This constraint ensures that all values in a column are different.

3. `PRIMARY KEY`: This constraint uniquely identifies each record in a database table. Primary keys must contain unique values. A primary key column cannot have NULL values.

4. `FOREIGN KEY`: This constraint is used to prevent actions that would destroy links between tables. A foreign key in one table points to a primary key in another table.

5. `CHECK`: This constraint ensures that all values in a column satisfy certain conditions.

6. `DEFAULT`: This constraint sets a default value for a column when no value is specified.

### CREATE INDEX

The CREATE INDEX statement is used to create indexes in tables.

Indexes are used to retrieve data from the database more quickly than otherwise. The users cannot see the indexes, they are just used to speed up searches/queries.



Note: Updating a table with indexes takes more time than updating a table without (because the indexes also need an update). So, only create indexes on columns that will be frequently searched against.

DROP INDEX Statement

The DROP INDEX statement is used to delete an index in a table.



### AUTO INCREMENT

AUTO INCREMENT Field
Auto-increment allows a unique number to be generated automatically when a new record is inserted into a table.

Often this is the primary key field that we would like to be created automatically every time a new record is inserted.

### CREATE VIEWS

In SQLite, a view is a virtual table based on the result-set of an SQL statement. A view contains rows and columns, just like a real table. The fields in a view are fields from one or more real tables in the database.

Here is an example of how to create a view:

```sql
CREATE VIEW view_name AS
SELECT column1, column2, ...
FROM table_name
WHERE condition;
```

For example, if you have a table named `Employees` and you want to create a view that only shows the `EmployeeName` and `Salary`, you could do:

```sql
CREATE VIEW Employee_Salaries AS
SELECT EmployeeName, Salary
FROM Employees;
```

After the view has been created, you can query it just like you would a normal table:

```sql
SELECT * FROM Employee_Salaries;
```

Remember to replace `view_name`, `column1`, `column2`, `table_name`, `condition`, `EmployeeName`, and `Salary` with your actual view name, column names, table name, condition, and column names respectively.

### SQL Injection

GitHub Copilot: Para protegerse de SQL Injection en SQLite con Python, se debe utilizar la característica de parámetros de consulta parametrizados que proporciona la biblioteca sqlite3. Esto asegura que los valores que se insertan en la consulta se escapen correctamente y no se puedan usar para inyectar SQL adicional.

Aquí hay un ejemplo de cómo hacerlo:

```python
import sqlite3

# Conectar a la base de datos SQLite
con = sqlite3.connect('my_database.db')

# Crear un objeto cursor
cur = con.cursor()

# Valor de entrada proporcionado por el usuario
user_provided_value = 'user input'

# Consulta SQL parametrizada
cur.execute("SELECT * FROM my_table WHERE my_column = ?", (user_provided_value,))

# Obtener los resultados
results = cur.fetchall()

# Cerrar la conexión
con.close()
```

En este código, `?` es un marcador de posición que se reemplaza de manera segura por el valor proporcionado por el usuario. Esto evita que el valor del usuario se interprete como SQL adicional, protegiendo contra la inyección de SQL.